# EDS Test Harness
This notebook can be used to test the EDS web services. It is interactive and users can modify it as they see fit. Modifcations to this notebook will be available to the user while they are actively using the notebook, they are not saved for future sessions.

Instructions

**1: Setup**
* Enter the client key that should be used for all requests
* Enter the api key that should be used for all requests
* Use the Run button above to set the key

**2: Get Data Catalog**
* Run the 'Get Data Catalog' box by pressing the Run button again

**3: Request Configuration**
* Edit the variables in the 'Get Data Request Configuration' box to specify an EDS data request. Use the results from the 'Get Data Catalog' request to set the 'dataset_id' and 'source' variables. The source is the header text for each dataset. The 'Get Data Request' box contains a default request for HYCOM_GLOBAL data that can be run as an example
* Run the 'Get Data Request Configuration' Box

**4: Get Data Request**
* Run the 'Get Data Request' box. This code will make a GetData request to EDS. When the file name is returned, it then makes a GetStatus request to EDS until a status of 'COMPLETE' is returned. When GetStatus returns 'COMPLETE', a Downloadfile request is made and a netCDF file is downloaded to the cloud machine running this notebook. A link will be displayed to the user that they can click to download the file from the cloud to their local machine.
* Click on the link to download the netCDF file to your local machine

In [ ]:
# Test Configuration Box

import requests
import json
from time import sleep

client_key = ""
api_key=""
eds_rest_url="https://coastmap.com/edsrest/"

In [ ]:
# Get Data Catalog Box
# Gets the Data Catalog from EDS 

catalog_req = requests.get(f"{eds_rest_url}datacoverage/{client_key}", 
           headers={'api-key':api_key}, params={'resultType':'Normal'})
data = json.loads(catalog_req.text)
for source in data:
    print(source['source'])
    print(f"   ID: {source['id']}")
    print(f"   Name: {source['name']}")
    print(f"   Description: {source['description']}")
    print(f"   Time span: {source['starttime']}-{source['endtime']}")
    print(f"   Longitude extents: {source['extentleft']}-{source['extentright']}")
    print(f"   Latitude extents: {source['extentbottom']}-{source['extenttop']}")

In [ ]:
# Get Data Request Configuration Box
# Define the request - Change values
# Get the ID and source from the data catalog results above
dataset_ID = 696 
source = "HYCOM_GLOBAL"
lon_min = -80.5
lon_max = -80
lat_min = 30.5
lat_max = 40.0
start_time = '2025-09-12 06:00'
end_time = '2025-09-14 03:00'

In [ ]:
# Get Data Request Box
# Make a request to EDS to download the data for the request 
data_req = requests.post(f"{eds_rest_url}/data",json ={"ClientKey": client_key,
       "SourceId" : dataset_ID,
       "SourceName": source,
       "OutputType" : 0,
       "StartDate" : start_time,
       "EndDate" : end_time,
       "X1" : lon_min,
       "X2" : lon_max,
       "Y1" : lat_min,
       "Y2": lat_max}, headers = {'api-key':api_key,'Content-Type':'application/json'})
req_id = json.loads(data_req.text)
print(f"Request ID: {req_id}")
while(1):
   status_req = requests.get(f"{eds_rest_url}orders/status/{req_id}",headers = {'api-key':api_key})
   status = json.loads(status_req.text)
   if("IN_PR" not in status): break
   sleep(2)
if status != "ERROR":
   nc_req = requests.get(f"https://coastmap.com/edsoutput/{req_id}")
   with open(req_id,"wb") as nc_f:
       nc_f.write(nc_req.content)
   print(f"Downloaded file: {req_id}")